# Part 1 of the Machine Learning Project

## Preliminaries


Before satring familiarize yourself with pandas reading the “10 minutes to pandas” tutorial: https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html

Browse through the full pandas user guide when needed: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html


## Load libraries

Test on Markdown

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# File system manangement
import os

# Suppress warnings
#import warnings
#warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


<p><img alt="Datasets" src="https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png" align="left" hspace="10px" vspace="0px"></p>


<h1>Datasets</h1>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
## Change this part with your own dataset files
previous_application = 'drive/MyDrive/Advanced_Machine_Learning/previous_application.csv'
src_bureau = 'drive/MyDrive/Advanced_Machine_Learning/bureau.csv'
src_bureau_balance = 'drive/MyDrive/Advanced_Machine_Learning/bureau_balance.csv'
src_train = 'drive/MyDrive/Advanced_Machine_Learning/application_train.csv'
src_test = 'drive/MyDrive/Advanced_Machine_Learning/application_test.csv'


# If you cannot load the directly the csv from Google drive (Google restrictions), download them manually then change the path to load them locally



<h1>Loading the datasets (2 pts)</h1>

1. Similarly to the train set, load the test set, the bureau dataset and the past applications dataset.

2. Display for each the number of rows and the number of columns

*You will be evaluated as follow*
* 1: Correct approach + code: 1pt
* 2: Code: 1pts


In [60]:
#Your code here
#The variables that should be implemented to store the data are: app_train, app_test, bureau, app_past


# 1 - Loading the different datasets.
prev_app = pd.read_csv(previous_application)
bureau = pd.read_csv(src_bureau)
bureau_balance = pd.read_csv(src_bureau_balance)
app_train = pd.read_csv(src_train)
app_test = pd.read_csv(src_test)

# 2 - The number of rows and columns for each
print(f'Number of rows and columns for previous application: {prev_app.shape}')
print(f'Number of rows and columns for bureau: {bureau.shape}')
print(f'Number of rows and columns for previous balance: {bureau_balance.shape}')
print(f'Number of rows and columns for train: {app_train.shape}')
print(f'Number of rows and columns for test: {app_test.shape}')


Number of rows and columns for previous application: (1670214, 37)
Number of rows and columns for bureau: (1716428, 17)
Number of rows and columns for previous balance: (27299925, 3)
Number of rows and columns for train: (307511, 122)
Number of rows and columns for test: (48744, 121)


<h1> Feature Engineering</h1>

<h2>Missing values (9pts)</h2>


**3.a**: What columns are missing the most values in app_test?

**3.b**: What columns are missing the most values in bureau?

**3.c**: What columns are missing the most values in app_past?

4: Fix missing data in app_test dataset using categorization and/or simple imputation when appropriate. Be careful to the **data leakage** issue!

*You will be evaluated as follow*
* 3: Correct approach + code 2pts.
* 4: Description of the approach 4pts. Code implementation of the approach 3pts.

In [61]:
# 3a missing values on app_test
missing_values = app_test.isnull().sum()
print(f'Columns with the most missing values: \n{missing_values.sort_values(ascending=False).head(5)}')

# 3b missing values on bureau
missing_values = bureau.isnull().sum()
print(f'Columns with the most missing values: \n{missing_values.sort_values(ascending=False).head(5)}')

## 3c
missing_values =  prev_app.isnull().sum()
print(f'Columns with the most missing values: \n{missing_values.sort_values(ascending=False).head(5)}')


Columns with the most missing values: 
COMMONAREA_AVG              33495
COMMONAREA_MODE             33495
COMMONAREA_MEDI             33495
NONLIVINGAPARTMENTS_AVG     33347
NONLIVINGAPARTMENTS_MODE    33347
dtype: int64
Columns with the most missing values: 
AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
dtype: int64
Columns with the most missing values: 
RATE_INTEREST_PRIVILEGED    1664263
RATE_INTEREST_PRIMARY       1664263
AMT_DOWN_PAYMENT             895844
RATE_DOWN_PAYMENT            895844
NAME_TYPE_SUITE              820405
dtype: int64


In [82]:
# 4: Fix missing data in app_test using categorization and/or simple imputation when appropriate. You should use *only* pandas and numpy here. You should *not* use sklearn (scikit-learn).

## Your code here

## Get the column names to iterate through them
columns_names = app_test.columns.tolist()

## Fill NaN numerical columns whith MEAN value
for column in columns_names:
  if app_test[column].dtype == 'float64' or app_test[column].dtype == 'numerical' or app_test[column].dtype == 'int':
    app_test[column] = app_test[column].fillna(app_test[column].mean())


## Fill NaN categorical columns with "Not specified" value
categorical_missing = app_test.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
categorical_missing = categorical_missing[categorical_missing > 0]
categorical_missing = categorical_missing.index.tolist()

for column in categorical_missing:
  app_test[column] = app_test[column].fillna('not specified')


app_test.isnull().sum().sort_values(ascending=False)




,0
SK_ID_CURR,0
COMMONAREA_MODE,0
EMERGENCYSTATE_MODE,0
WALLSMATERIAL_MODE,0
TOTALAREA_MODE,0
...,...
REG_CITY_NOT_LIVE_CITY,0
LIVE_REGION_NOT_WORK_REGION,0
REG_REGION_NOT_WORK_REGION,0
REG_REGION_NOT_LIVE_REGION,0


**4**: Describe your approach, including how you choose the technique to apply, and how you apply the technique (double click to edit a text cell)



## Class Imbalance (8 pts)

5. Evaluate the class imbalance of the training set.

6. **a** Fix the class imbalance with over/undersampling

 **b** Use the SMOTE algorithm to fix class imbalance

*You will be evaluated as follow*
* 5: Correct approach + code: 2ts
* 6.a: code 3pt
* 6.b: code 3pts

In [ ]:
# 5 - value counts


# 5 - histograms


**5**: Describe briefly your observation (double click to edit a text cell)

In [ ]:
# copy your current dataframe with its .copy() method. You should always use the copy() method if you want to keep the original dataframe untouched when you modify
# this is a very common bug, so please read  https://www.statology.org/pandas-copy-dataframe/

## Your code here
app_train_original = ...


# 6.a - fix imbalance with undersampling or oversampling
# Implement undersampling or oversampling *without* external libraries, only the library provided to implement yourself the chosen solution.

## Your code here


# 6.b - fix balance with SMOTE on your previously copied dataframe
# You can use any external libraries to use SMOTE. We recommend the imbalanced-learn package https://imbalanced-learn.org/stable/over_sampling.html
# Make sure to select the appropriate SMOTE variant.


## Your code here
app_train_smote = ...


# We will use the dataframe app_train from 6.a in the following

<h2>Categorical features (9pts)</h2>

In `app_test`

**7a**: How will you transform the column NAME_HOUSING_TYPE in app_test be correctly handled by the model? How will you transform CODE_GENDER column?

**7b**: How would you transform the column NAME_HOUSING_TYPE in app_test if you want only 3 categories? Use LabelEncoder to transform FLAG_OWN_CAR column.

**7c**: In ORGANIZATION_TYPE, only keep the categories that appear in more than 10% of the dataset. Group other categories in one. What is the size of the dummy vector for ORGANIZATION_TYPE after this transformation?


8: **In app_test, app_past and bureau,** transform all the categorical columns that you have not processed in 7 using One-hot encoding.

*You will be evaluated as follow*
* 7: Description of the approach 4pts. Code 3pts.
* 8: Correct approach + code 2pts.

In [ ]:
## Your code here

<h2>Anomalies (7 pts)</h2>

**In app_test**

**9a**: Is there an anomaly with the column DEF_30_CNT_SOCIAL_CIRCLE? If so, how would you solve it?

**9b**: Is there an anomaly with the column LANDAREA_AVG? If so, how would you solve it?

**9c**: Is there an anomaly with the column AMT_INCOME_TOTAL? If so, how would you solve it?

10: Fix the anomaly in DAYS_EMPLOYED?

*You will be evaluated as follow*
* 9: Description of the approach + code 5pts.
* 10: code 2pts

In [ ]:
## Your code here

<h2>Correlations (9 pts)</h2>


11: **In app_test**, remove the collinear features of the dataset. How did you choose the threshold?

12: What features from bureau.csv could you use to improve the training set? Use the random forest model from the notebook of the Lecture to check if it actually improves the final model performance.

*You will be evaluated as follow*
* 11: Code 3pts. Threshold explanation 1pt.
* 12: Description of the approach 1pt. Code implementation of the approach 2pts. Improved performance 2pts.

In [ ]:
## Your code here

##  Class imbalance and model performances (6 points)


**13**: Evaluate the other technique (SMOTE) to handeling imbalance data left in *6.b*. Does it makes a difference on the final model performance? Can you use the performance on the test set to choose which techniques to use? If not, what would you need to do so?

*You will be evaluated as follow*
* 13: Code 3pts. Explanation 3pts.


In [ ]:
## Your code here

**13**: Answer the question here (double click to edit a text cell)

